In [209]:
import numpy as np
import xlrd # xlrd is a library for reading data and formatting information from Excel files in the historical .xls format

In [210]:
path = 'data_combined.xls'

# hh is the number of rows in the header, nn is the number of columns in the header
hn, nc = 1, 4

In [211]:
# function to read data from Excel file
def readexcel(hn, nc):
    data = xlrd.open_workbook(path)
    table = data.sheet_by_name(sheetname)
    nrows = table.nrows
    data = []
    column_names = []
    for i in range(hn, nrows):
        data.append(table.row_values(i)[nc:])
    
    for i in range(hn):
        column_names.append(table.row_values(i)[nc:])
        
    return column_names, np.array(data)

In [212]:
def entropy(data):
    # In this method, m indicators and n samples are set in the evaluation, and the measured value
    # of the i_th indicator in the j_th sample is recorded as x_ij.
    n, m = len(data), len(data[0])
    
    
    # if negative values present, make the column all positive number, by adding minim_column value
    for i in range(m):
        minim = 99999
        for j in range(n):
            if data[j][i] < minim:
                minim = data[j][i]
        if minim < 0:
            for j in range(n):
                data[j][i] += abs(minim)
        
    
    # the first step is the standardization of measured values
    # the standardized value of the i_th index in the j_th sample is denoted as p_ij, and its calculation
    # method is as follows:
    # p_ij = (x_ij) / (sum(j = 1 -> n) of x_ij)
    
    data_column_sum = [sum(column) for column in zip(*data)]
    
    normalized_data = [[data[j][i] / data_column_sum[i] for i in range(m)] for j in range(n)]
    
    # In the EWM, the entropy value E_i of i_th index is defined as
    # E_i = - (sum(j = 1 -> n) p_ij * ln p_ij) / (ln n)
    # In the actual evaluation using the EWM, p_ij * ln p_ij = 0 
    # is generally set when p_ij = 0 for the convenience of calculation
    
    normalized_data_column = []
    
    for i in range(m):
        inter_sum = 0
        for j in range(n):
            if normalized_data[j][i] == 0:
                inter_sum = 0
            else:
                inter_sum += normalized_data[j][i] * np.log(normalized_data[j][i]) 
        normalized_data_column.append(-inter_sum)
    
    entropy_values = [x / np.log(n) for x in normalized_data_column]
        
    # The range of entropy value E_i is [0, 1]. The larger the E_i is, the greater
    # the differantiation degree of index i is, and more information can be derived.
    # Hence higher weight should be given to the index. Therefore, in the EWM, the calculation
    # method of weight w_i is w_i = (1 - E_i) / (sum(i = 1 -> m)(1 - E_i))
    
    weights = [100 * (1 - E_i) / (m - sum(entropy_values)) for E_i in entropy_values]
    
    return weights

In [219]:
for sheetname in [u'Environmental', u'Social', u'Governance']:
    column_names, data = readexcel(hn, nc)
    weights = entropy(data)

    print('Sheetname: ', sheetname,)
    print('Column name, weight of column in percents')
    print(' ')
    print(*list(zip(column_names[0], weights)), sep = '\n')
    print('________________________________________________________________________ \n\n')

Sheetname:  Environmental
Column name, weight of column in percents
 
('Cereal_yield', 0.7770616857602388)
('Mean_area_protected_terrestrial ', 1.0750803784535703)
('Wastewater_Treatment ', 44.54139693490824)
('Electronic_waste ', 1.7304984769015659)
('CO2_emissions_combustion', 0.8385354377455345)
('Nitrogen_emissions ', 1.2151087405952323)
('CO2_emissions_exports', 49.42672784766029)
('Sustainable Nitrogen Management Index ', 0.3955904979752965)
________________________________________________________________________ 


Sheetname:  Social
Column name, weight of column in percents
 
('Poverty at $1.90/day', 21.870308036805156)
('Subjective_well_being', 0.054205562115789606)
('Unemployment ', 0.8251522643080216)
('Employment_to_population', 0.0776338918484052)
('Gini ', 0.11455276276311065)
('Internet_users', 0.2188564874001799)
('Mobile_subscriptions ', 0.3346395376966971)
('Net_primary_enrollment ', 0.010624336477206798)
('Lower_secondary_completion ', 0.05115317105672444)
('Female_t